<a href="https://colab.research.google.com/github/WetSuiteLeiden/example-notebooks/blob/main/research-methods/methods_nlp__concordance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal of this notebook

Explain what a concordance is, and how it might help you explore what a word or phrase means.

Give some basic code and basic examples

## What 

A concordance has some [varied meanings in different fields](https://en.wikipedia.org/wiki/Concordance), 
but in [publishing](https://en.wikipedia.org/wiki/Concordance_(publishing) and linguistics it usually means to taking a term or phrase,
and shows all occurences of that term in the context it is used in) - context of a few words, up to a sentence.

Before that could be automated this was only done for a few words of special importance. 
With computers we can do it arbitrarily, we just have to deal with the bulk of output.

No, this is not at all complex, but it can still be quite _useful_ to a few tasks, such as inspecting your data,
finding things to match, finding out how to write pattern rules, etc.


Yes, you can think up more features to use this for actual research.
    
Yes, you can potentially do this faster with an installation of a search engine.

Both of those represents a learning curve, and you can get a decent stat from a relatively small piece of DIY code, such as that below.

In [ ]:
# For local installs you can install the package once.   In colab you get a disposable environment and will have to start with this install each time. 
!pip3 install wetsuite -U

In [ ]:
!python -m spacy download nl_core_news_lg

In [2]:
import re
import random
import spacy
import wetsuite.helpers.localdata
import wetsuite.helpers.strings
import wetsuite.helpers.escape
import wetsuite.helpers.spacy
import wetsuite.helpers.notebook

In [3]:
# TODO: make a dataset like this so that you don't have to do it yourself

# a list of strings, made elsewhere
sentence_store = wetsuite.helpers.localdata.LocalKV('sentences.db', str, str)

# assuming the full thing is a _lot_... 
#sentences = sentence_store.values()
#...then around 50K seems a good balance while developing, between having some cases and not spending very long
sentences = sentence_store.random_values(100000)

In [4]:
dutch = spacy.load('nl_core_news_lg')

In [6]:
def diy_concordance( needle_re, haystack_strlist ):
    for hs_item in haystack_strlist:
        for before_str, match_str, match_obj, after_str in wetsuite.helpers.strings.findall_with_context(needle_re, hs_item, 40):
            # print it out in a way that makes it clear what part was the match
            print(f'{before_str:>50s} {match_str:^30s} {after_str}')


# the reason we chose regular expression is that, aside from literal matches such as...
#diy_concordance(r'verbaal', sentences)

# ...it allows (limited) tricks like "words ending in vergunning" (via "zero or more non-space character ending in the letters vegunning")
diy_concordance(r'[^\s]*vergunning\b', sentences)

# ...or words with 'minister' somewhere in it   (The (?i) is ap python-regex thing that makes it case insensitive)
#diy_concordance(r'(?i)[^\s]*minister[^\s]*', sentences)   
#diy_concordance(r'(?i)[^\s]+fraud[^\s]+\b', sentences)


          nationaliteit heeft, beschikte over een            vergunning            tot verblijf in Nederland zonder beperk
           in de weg stonden aan het verlenen van          bouwvergunning         .
                               De stelling dat de         milieuvergunning         inmiddels is vervallen, wat daar verder
           appellant waar blijkens de aanvraag om          bouwvergunning          afvalstoffen, bouwstoffen en containers
          k met verweerder van oordeel is dat een          bouwvergunning          noodzakelijk is.
           dat het aanvragen van de hier bedoelde            vergunning            in principe geboden is.
          heid niet heeft mogen weigeren haar een       omgevingsvergunning        voor het afwijken van het bestemmingspl
          n besluit heeft verweerder de gevraagde       omgevingsvergunning        verleend.
          ast, omdat sprake is van een vernieuwde            vergunning            voor dezelfde soort activiteiten en goe
 

Let's try to make something a little more controllable,
and give you the POS tags that you might indeed want when [developing rules](methods_nlp__patterns_spacy.ipynb).

In [74]:
from wetsuite.helpers.escape import nodetext

class Concordance():
    def __init__( self, haystack_items, context_charamt:int=30 ):
        ' haystack_items should be a list of strings, the things to search in later'
        self.haystack        = haystack_items
        self.context_charamt = context_charamt

    def process(self, needle_re):
        ''' Searches for a regular expression in all the items previously handed in.
            Returns a list of (string before, matching string, string after) '''
        self.matches = []
        for hs_item in self.haystack:
            for before_str, match_str, _match_obj, after_str in wetsuite.helpers.strings.findall_with_context(needle_re, hs_item, self.context_charamt):
                self.matches.append( (before_str, match_str, after_str) )
    
    def process_with_spacy(self, needle_re, model):
        ''' Search like process(), but also adds POS tags using spacy.  
            Same return tuple as process(), and adds them in HTML tags.
        '''
        # TODO: review, there is likely an off-by-1 error in here
        # CONSIDER: cacheing sentence parses
        self.matches = []
        for hs_item in self.haystack:
            for before_str, match_str, match_obj, after_str in wetsuite.helpers.strings.findall_with_context(needle_re, hs_item, self.context_charamt):

                m_st, m_en = match_obj.start(), match_obj.end()
                #print("Match at char pos %d..%d"%(m_st, m_en))
                doc = dutch(hs_item)
                toks = list(doc)

                before, hit, after = [],[],[]

                # tokens before match start (at m_st)
                while len(toks)>0: 
                    tok = toks.pop(0)
                    if tok.idx > m_st-self.context_charamt:
                        before.append('%s<span class="g">/%s</span>'%(nodetext(tok.text), nodetext(tok.pos_)))
                    if tok.idx + len(tok.text_with_ws) + 1 > m_st:
                        break
                # tokens before match end
                while len(toks)>0: 
                    tok = toks.pop(0)
                    if tok.idx+1 > m_en:
                        toks.insert(0, tok) # then put it back for the next block to pick up
                        break
                    hit.append('%s<span class="g">/%s</span>'%(nodetext(tok.text), nodetext(tok.pos_)))
                # tokens after match
                for tok in toks:
                    if tok.idx < m_en + self.context_charamt:
                        after.append('%s<span class="g">/%s</span>'%(nodetext(tok.text), nodetext(tok.pos_)))

                self.matches.append( (' '.join(before), ' '.join(hit), ' '.join(after)) )

    def _repr_html_(self):
        ' Take processed data and format for notebooks - as a HTML table '
        ret = ['<table>']
        for before, m, after in self.matches:
            ret.append('<tr>')
            ret.append(f'<td>{before}</td>')
            ret.append(f'<td class="c2">{m}</td>')
            ret.append(f'<td class="c3">{after}</td>')
            ret.append('</tr>')
        ret.append('</table>')
        #padding-left:2em; text-indent: -2em; 
        return '<div style="width:100%%"><style>.c2 {text-align:center; color:green} .c3 {text-align:left} .g {opacity:0.33;}</style>%s</div>'%"".join(ret)
    
c = Concordance( sentences )
#c.process(r'[^\s]*vergunning\b' )                       # no POS
c.process_with_spacy(r'(?i)[^\s]+fraud[^\s]+\b', dutch ) # with POS
display( c )

“/PUNCT /SPACE in/ADP Europa/PROPN op/ADP grote/ADJ schaal/NOUN,btw-fraude/NOUN,plaatsvindt/VERB met/ADP zogeheten/ADJ koper/NOUN
De/DET term/NOUN,btw-carrouselfraude/ADJ,wordt/AUX gebruikt/VERB voor/ADP een/DET samenstel/NOUN
hiermee/ADV dat/SCONJ eisende/ADJ partij/NOUN,gefraudeerd/VERB,heeft/AUX ./PUNCT
dat/DET kader/NOUN gezamenlijk/ADJ hebben/AUX,gefraudeerd/VERB,"met/ADP PGB-gelden/NOUN ,/PUNCT door/ADP het/DET innen/VERB"
is/AUX het/DET Centraal/PROPN Meldpunt/PROPN,Faillissementsfraude/PROPN,ondergebracht/VERB ./PUNCT
",/PUNCT in/ADP verband/NOUN gebracht/VERB met/ADP",beleggingsfraude/NOUN,en/CCONJ het/PRON niet/ADV nakomen/VERB van/ADP verplichtingen/NOUN
ontvangen/VERB over/ADP mogelijke/ADJ,beleggingsfraude/NOUN,door/ADP beleggingsvereniging/NOUN “/PUNCT Fibonacci/NOUN
"geschrift/NOUN (/PUNCT feiten/NOUN 5/NUM t/m/ADJ 10/NUM )/PUNCT ,/PUNCT",faillissementsfraude/PROPN,(/PUNCT feit/NOUN 11/NUM )/PUNCT en/CCONJ onttrekking/NOUN aan/ADP
een/DET proces-verbaal/NOUN van/ADP,uitkeringsfraude/NOUN,van/ADP de/DET Dienst/PROPN Werk/NOUN en/CCONJ Inkomen/VERB
verwijt/NOUN dat/PRON [/NOUN gedaagde/VERB ]/PROPN heeft/AUX,gefraudeerd/VERB,ter/ADP zitting/NOUN nog/ADV verklaard/VERB dat/SCONJ
is/AUX neergelegd/VERB in/ADP een/DET Rapport/NOUN,Uitkeringsfraude/NOUN,van/ADP 22/NUM november/PROPN 2002/NUM ./PUNCT


One way you might extend this is to 
- get data out as python data, dataframe, etc.
- mention the source of each thing we are searching in, so that we can trace it back.

<!--

further reading

https://spotintelligence.com/2022/11/28/tf-idf/

https://www.tutorialspoint.com/gensim/gensim_creating_tf_idf_matrix.htm

https://mayurji.github.io/blog/2021/09/20/Tf-Idf

-->